In [2]:
import pandas as pd
import pickle

In [97]:
df = pd.read_pickle('data_clean.pkl')

In [114]:
models = pd.read_csv('models.csv', header=None, names = ['brand', 'model'])
models['brand'] = models['brand'].str.replace(r'[^\w\s]+', '', regex=True)

In [115]:
models

brand    model
0    acura  integra
1    acura   Legend
2    acura    vigor
3    acura      rlx
4    acura      ILX
..     ...      ...
523  volvo     xc90
524  volvo      s60
525  volvo      s80
526  volvo      v60
527  volvo      c70

[528 rows x 2 columns]

In [116]:
df.Message.str.contains('said').sum()

370

In [356]:
df.Message_words = df.Message_words.apply(lambda x: [i.lower() for i in x])

### Replace models with brands

In [357]:
messages = df.Message_words.to_list()

In [121]:
# numpy.where to do the replacement
messages2 = []
t = 0
for m in messages:
    for i,row in models.iterrows():
        m = np.where(m == row['model'], row['brand'], m)
        
    t+=1
    
    #print(t)
        
    messages2.append(m)
        
df['Message_words_v2'] = messages2

In [122]:
# let's find the occurences of changes
m1 = df.Message_words.to_list()
m2 = df.Message_words_v2.to_list()

c = 0
W = 0
for i in range(len(m2)):
    for j in range(len(m2[i])):
        if m1[i][j] != m2[i][j]: #word by word comparison
            c +=1
        W += 1

In [123]:
print(f'{c} changes of models')

8274 changes of models


## Link attributes to models

In [ ]:
# first let tag all records that only have 1 brand mention

In [265]:
brands_list = models.brand.drop_duplicates().to_list()

In [266]:
brands_list.remove('car')
brands_list.remove('sedan')
brands_list.remove('problem')


In [267]:
messages = df.Message_words_v2.to_list()

In [268]:

brands_in_message = []
for m in messages:
    
    brands_mentioned = []
    for brand in brands_list:
        if len(np.where(m == brand)[0]) >0:
            brands_mentioned.append(brand)
            
    brands_in_message.append(brands_mentioned)
        
    

In [269]:
df['Brands_in_message'] = brands_in_message

In [270]:
df['Qty_brands'] = df.Brands_in_message.apply(lambda x: len(x))

In [271]:
df['Qty_brands'].value_counts()

# Most messages discuss a single brand
# Also common to compare 2 brands
# 727 messages don't mention any brand (remove them from analysis?)
# Passing a certain treshold (5+), I think it would be better to remove comments since it will be hard to figure out which
# attribute belongs to each brand

0     1291
1     1215
2      755
3      459
4      219
5      109
6       54
7       24
8       10
9        6
11       4
10       3
16       1
13       1
Name: Qty_brands, dtype: int64

In [272]:
pd.set_option('display.max_colwidth', None)

In [273]:
df.loc[df.Qty_brands == 0, 'Message_words_v2'].head(10)

0                                                                                      [need, help, choosing, next, vehicle, tell, us, price, range, thinking, new, used, buying, leasing, features, musthaves, thoughts, mind, let, us, give, hand, tell, us, criteria, specifically, canyoull, find, lots, helpful, folks, give, useful, suggestions]
2     [buy, 2001, 2002, oldsmobile, intrigue, one, best, sedan, market, great, handling, one, best, v6, engines, around, 35l, dohc, producing, 215hp, deals, great, year, last, intrigue, warrany, extended, 5, years, car, recommended, magazines, including, consumer, reports, unfortunately, looked, consumers, sleeper, sleepers, market, opinion]
8                                                                                                                                                                                                                                                                                     [id, pass, intrigue, one, reason, 

In [274]:
m = df.Message_words_v2[1]

In [275]:
m

array(['im', 'buying', 'new', 'car', 'end', 'year', 'first', 'choice',
       'nissan', 'nissan', 'ive', 'read', 'reliable', 'hold', 'value',
       'well', 'budget', 'probably', 'going', 'allow', '9899',
       'wondering', 'worth', 'buying', 'car', 'thats', 'almost', '34',
       'years', 'old', 'suggestions', 'comments', 'would', 'help',
       'lotwillow25'], dtype='<U11')

In [276]:
len(np.where(m == 'helpfull')[0])

0

In [277]:
brands_list

['acura',
 'audi',
 'bmw',
 'buick',
 'cadillac',
 'chevrolet',
 'chrysler',
 'dodge',
 'ford',
 'honda',
 'hyndai kia',
 'hyundai',
 'infiniti',
 'kia',
 'lincoln',
 'mazda',
 'mercedes',
 'mercury',
 'mitsubishi',
 'nissan',
 'pontiac',
 'saturn',
 'seat',
 'subaru',
 'suzuki',
 'toyata',
 'toyota',
 'volkswagen',
 'volkwagen',
 'volvo']

In [286]:
m = df.Message_words_v2[9] #record 9 is a good exaple
brand_dic = {}
for brand in brands_list:
    ix = np.where(m == brand)[0]
    if len(ix) > 0:
        brand_dic[brand] = ix[0]
        

brand_dic = dict(sorted(brand_dic.items(), key=lambda x:x[1]))

In [ ]:
# we can remove duplicate words in the early preprocessing

In [335]:
brand_list = list(brand_dic.values()) + [len(m)]

for i, tup in enumerate(brand_dic):
    print(brand_dic[tup])
    print(tup)
    
    if i == 0:
        
        print(m[0:brand_list[1]])
    else:
        print(m[brand_list[i]:brand_list[i+1]])
        
        

2
honda
['would' 'recommend' 'honda' 'honda']
4
toyota
['toyota' 'toyota' 'choices' 'sedan' 'new']
9
nissan
['nissan' 'nissan' 'looks' 'impressive' 'would' 'wait' 'reliability'
 '2002' 'proventhe' 'new']
19
bmw
['bmw' '525' 'another' 'nice' 'car' 'rated' 'highly' 'many' 'auto'
 'publications']


In [358]:
brand_reviews = []

for m in df.Message_words_v2:

    brand_dic = {}
    for brand in brands_list:
        ix = np.where(m == brand)[0]
        if len(ix) > 0:
            brand_dic[brand] = ix[0]

    brand_dic = dict(sorted(brand_dic.items(), key=lambda x:x[1]))

    brand_list = list(brand_dic.values()) + [len(m)]


    review={}
    for i, tup in enumerate(brand_dic):
        #print(brand_dic[tup])
        #print(tup)

        if i == 0:
            
            review[tup] = m[0:brand_list[1]]
        else:
            review[tup] = m[brand_list[i]:brand_list[i+1]]
            
    brand_reviews.append(review)

# generates a list (1 entry per row) of dictionaries {brand: part of text corresponding}

In [359]:
len(brand_reviews)

4151

In [355]:
brand_reviews[0]

{'honda': array(['would', 'recommend', 'honda', 'honda'], dtype='<U12'),
 'toyota': array(['toyota', 'toyota', 'choices', 'sedan', 'new'], dtype='<U12'),
 'nissan': array(['nissan', 'nissan', 'looks', 'impressive', 'would', 'wait',
        'reliability', '2002', 'proventhe', 'new'], dtype='<U12'),
 'bmw': array(['bmw', '525', 'another', 'nice', 'car', 'rated', 'highly', 'many',
        'auto', 'publications'], dtype='<U12')}

In [373]:
brands_list

['acura',
 'audi',
 'bmw',
 'buick',
 'cadillac',
 'chevrolet',
 'chrysler',
 'dodge',
 'ford',
 'honda',
 'hyndai kia',
 'hyundai',
 'infiniti',
 'kia',
 'lincoln',
 'mazda',
 'mercedes',
 'mercury',
 'mitsubishi',
 'nissan',
 'pontiac',
 'saturn',
 'seat',
 'subaru',
 'suzuki',
 'toyata',
 'toyota',
 'volkswagen',
 'volkwagen',
 'volvo']

In [375]:
review_accum = {i:[] for i in brands_list}

In [376]:
for brand in brands_list:
    for review in brand_reviews:
        for single_review in review:
            #print(review[single_review])
            if single_review==brand:
                review_accum[brand].extend(review[single_review])

In [379]:
review_accum

{'acura': ['acura',
  'come',
  'v8',
  'model',
  'acura',
  'rl',
  'might',
  'try',
  'acura',
  'tl',
  'lexus',
  'es300',
  'smallest',
  'caddy',
  'rwd',
  'ctsid',
  'pick',
  'acura',
  'tl',
  'type',
  'best',
  'value',
  'edmunds',
  'editors',
  'choice',
  'well',
  'lexus',
  'worth',
  'look',
  'since',
  'want',
  'quieter',
  'car',
  'dont',
  'remember',
  'exactly',
  'going',
  'know',
  'got',
  'ton',
  'bad',
  'press',
  'sully',
  'name',
  'bit',
  'kind',
  'like',
  'isuzu',
  'trooper',
  'flipping',
  'killed',
  'acura',
  'version',
  'remember',
  'current',
  'grand',
  'cherokee',
  'sudden',
  'acceleration',
  'im',
  'looking',
  'buying',
  'new',
  'family',
  'sedan',
  'near',
  'luxury',
  'class',
  'looking',
  '1',
  'acura',
  'tl',
  '32',
  'type',
  '2',
  '02',
  'acura',
  'acura',
  'great',
  'acura',
  'well',
  'sporty',
  'nissan',
  'upgrade',
  'honda',
  'think',
  'claywater',
  'godeacs',
  'dead',
  'car',
  'near',
 